In [2]:
import numpy as np

from src.algorithm_functions import *
import random

In [164]:
train_df, test_array = split_ratings('Datasets/ratings.csv')
train_array = np.array(train_df)

In [171]:
def perform_svd22(na_indx, train_array: np.ndarray, test_array: np.ndarray, r: int,
                      max_iter: int = 100, min_diff: float = 0.0089):

    Z_i = copy.deepcopy(train_array)
    m = copy.deepcopy(train_array[~na_indx])
    i = 0
    diff = 10 ** 5

    while (i < max_iter) & (min_diff < diff):
        Z_i[~na_indx] = np.array(m).reshape(-1)
        svd = TruncatedSVD(n_components=r)
        svd.fit(Z_i)
        Sigma2 = np.diag(svd.singular_values_)
        VT = svd.components_
        W = svd.transform(train_array) / svd.singular_values_
        H = np.dot(Sigma2, VT)
        Z_ii = np.dot(W, H)
        diff = ((Z_ii - Z_i) ** 2).sum() / (Z_ii.shape[0] * Z_ii.shape[1])
        Z_i = copy.deepcopy(Z_ii)
        i += 1

    return Z_ii

In [174]:
train_array = fillna_means_weighted(train_df, 0.25)
a = perform_svd22(train_df.isna(), train_array, test_array, r=8, min_diff=0.0086)

In [176]:
calc_rmse(test_array, a)

0.8638018475024429

In [182]:
a[a > 5] = 5.0
b = np.round(a)
calc_rmse(test_array, b)

0.9121871967702717

In [131]:
lada = 0.1
r = 30

n, d = train_array.shape
eta = 0.02
min_diff = 1e-9
max_iter = 10000
W = np.matrix(np.full((n, r), 2/np.sqrt(r)), dtype=np.longdouble)
H = np.matrix(np.full((r, d), 2/np.sqrt(r)), dtype=np.longdouble)
possible_ix = np.argwhere(train_df.notnull().values).tolist()

In [132]:
W_prev = copy.deepcopy(W)
H_prev = copy.deepcopy(H)
possible_ix = np.argwhere(train_df.notnull().values).tolist()
diff_W = min_diff + 1
diff_H = min_diff + 1
rmse_prev = 1000
rmse = 0
iter = 0

while (iter < max_iter) & (min_diff < diff_W) & (min_diff < diff_H):
    ix = random.sample(range(0, len(possible_ix)), 1)
    i, j = possible_ix[ix[0]]
    grad_w = -2 * ((train_array[i, j] - np.float64(W[i, :] * H[:, j])) * H[:, j]).T + 2 * lada * W[i, :]
    grad_h = -2 * ((train_array[i, j] - np.float64(W[i, :] * H[:, j])) * W[i, :]).T + 2 * lada * H[:, j]
    W[i, :] = W[i, :] - eta * grad_w
    H[:, j] = H[:, j] - eta * grad_h
    diff_W = np.abs(W - W_prev).sum() / (n * r)
    diff_H = np.abs(H- H_prev).sum() / (r * d)
    W_prev = copy.deepcopy(W)
    H_prev = copy.deepcopy(H)
    iter += 1

Z_tilde = np.dot(W, H)
calc_rmse_longdouble(test_array, Z_tilde)

2.0296735096464884

In [133]:
iter

1877

In [163]:
global_mean = train_df.mean().mean()
W = np.matrix(np.full((n, r), np.sqrt(global_mean) / np.sqrt(r)), dtype=np.longdouble)
H = np.matrix(np.full((r, d), np.sqrt(global_mean) / np.sqrt(r)), dtype=np.longdouble)
a = perform_sgd(train_df, test_array, W, H, 80, 0.1, 0.01)
a

(1.0142747145371265, 1758)

In [9]:
(min_diff < diff_W) & (min_diff < diff_H)

In [150]:
def perform_sgd(train_df, test_array, init_W, init_H, r: int, lada: float, learning_rate: float,
                max_iter: int = 10000, min_diff: float = 5e-10):

    train_array = np.array(train_df)
    n, d = train_array.shape
    W = np.matrix(init_W, dtype=np.longdouble)
    H = np.matrix(init_H, dtype=np.longdouble)
    W_prev = copy.deepcopy(W)
    H_prev = copy.deepcopy(H)
    possible_ix = np.argwhere(train_df.notnull().values).tolist()
    diff_W = min_diff + 1
    diff_H = min_diff + 1
    iter = 0

    while (iter < max_iter) & (min_diff < diff_W) & (min_diff < diff_H):
        ix = random.sample(range(0, len(possible_ix)), 1)
        i, j = possible_ix[ix[0]]
        grad_w = -2 * ((train_array[i, j] - np.float64(W[i, :] * H[:, j])) * H[:, j]).T + 2 * lada * W[i, :]
        grad_h = -2 * ((train_array[i, j] - np.float64(W[i, :] * H[:, j])) * W[i, :]).T + 2 * lada * H[:, j]
        W[i, :] = W[i, :] - learning_rate * grad_w
        H[:, j] = H[:, j] - learning_rate * grad_h
        diff_W = np.abs(W - W_prev).sum() / (n * r)
        diff_H = np.abs(H - H_prev).sum() / (r * d)
        W_prev = copy.deepcopy(W)
        H_prev = copy.deepcopy(H)
        iter += 1

    Z_tilde = np.dot(W, H)
    rmse = calc_rmse_longdouble(test_array, Z_tilde)

    return rmse, iter